# Evolve-Filter, time dep, N varies

* $\delta = 1/nx$
* $a = a_D$

In [1]:
notes = "\n Evolve then Filter: BE using P2 and halving timestep at each refinement level \n\n"

from dolfin import *
from __future__ import print_function
import numpy as np
import sympy as sym
import csv

x, y, t = sym.symbols('x[0], x[1], t')
sigma = 1.0
mu = 10**(-5) 
b=as_vector([2.0, 3.0])

# Iliescu Exact Solution
c = 16.0*sym.sin(sym.pi*t)
h = x*(1-x)*y*(1-y)
g = 2*mu**(-0.5)*(0.25**2 - (x - 0.5)**2 - (y - 0.5)**2 )
ue = c*h*(0.5+sym.atan(g)/sym.pi)

# ------------------------------------------ #

ue = sym.simplify(ue)
u_code = sym.printing.ccode(ue)
u_code = u_code.replace('M_PI','DOLFIN_PI')

# du/dt - mu*Laplace(u) + div(bu) + sigma*u = f
fe = sym.diff(ue,t)
fe += - mu*(sym.diff(sym.diff(ue,x),x) + sym.diff(sym.diff(ue,y),y))
fe += b[0]*sym.diff(ue,x) + b[1]*sym.diff(ue,y)
fe += sigma*ue

f_code = sym.printing.ccode(fe)
f_code = f_code.replace('M_PI','DOLFIN_PI')

# print('u_code = ' + u_code + '\n')
# print('f_code = ' + f_code)

def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	return L2n, H1n

def compute_extrema(u, t):
    maxval = np.amax(u.vector().get_local())
    minval = np.amin(u.vector().get_local())
    return maxval, minval

# filter solving

def deconv_filter(u_, delta, Q, boundary):
    u_tilde = TrialFunction(Q)
    v = TestFunction(Q)

    F_Hfilter = v*u_tilde*dx + delta*delta*dot(grad(v), grad(u_tilde))*dx - v*u_*dx 

    a_Hfilter = lhs(F_Hfilter)
    L_Hfilter = rhs(F_Hfilter)
    
    bc = DirichletBC(Q, u_, boundary)
    
    A_Hfilter = assemble(a_Hfilter)
    bc.apply(A_Hfilter)

    b_Hfilter = assemble(L_Hfilter)
    bc.apply(b_Hfilter)

    solver = LUSolver(A_Hfilter)
    u_tilde = Function(Q)
    solver.solve(u_tilde.vector(), b_Hfilter)
    
    return u_tilde

def EF_t(remark, N, delta, nx, dt, T, u_code, f_code, sigma, mu, velocity, chi):
    degree = 2
    folder = 'FEF_BQ/'

    
    t = 0.0
    u_exact = Expression(u_code, degree = degree+1, t = t)
    f = Expression(f_code, degree = degree+1, t = t)

    mesh = UnitSquareMesh(nx,nx)
    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression(u_exact.cppcode, degree = degree+1, t = t)
    
    def boundary(x, on_boundary):
        return on_boundary

    
# ------------ STEP 1: Solve on coarse grid ----------------------#

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
#     u_n = Function(Q)
    u_n = interpolate(u_D, Q)
    u_ = Function(Q)
    

    # Galerkin variational problem
    F = (u - u_n)*v*dx
    F += dt*(mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx)

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    # Assemble matrices
    A1 = assemble(a1)

    # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)

    # Outputting files
    #out_file_uexact = File(folder+"evolve_u_exact_"+str(nx)+".pvd") 
    #out_file_ubar = File(folder+str(mu)+"evolve_u_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    out_file_ind = File(folder+"EF_aD"+str(nx)+'_N'+str(N)+".pvd")
    out_file_ufilter = File(folder+"EF_u_"+str(nx)+'_N'+str(N)+".pvd") 

    # Save t = 0.0
    #out_file_uexact << (ue, float(t))
    
    
    u_.rename('u','u')
    indicator = Expression('0.0', degree=2, t=0.0)
    indicator = interpolate(indicator, Q)
    indicator.rename('a','a')
    
    #out_file_ufilter << (u_, float(t))
    #out_file_ind << (indicator, float(t))
    
    while t - T + dt < DOLFIN_EPS:
        # Step 1 Solve on Coarse Grid
        t += dt

        u_D.t = t
        f.t = t
        u_exact.t = t
        
        b = assemble(L)
        bc = DirichletBC(Q, u_D, boundary)
        bc.apply(A1)
        bc.apply(b)
        
        solve(A1, u_.vector(), b)#, 'gmres')
        progress.update(t / T)
        
#         if near(t,0.5):
#             errors = compute_errors(u_exact, u_, t, mesh)
#             print("Before Filter \n" + errors)

        #out_file_uexact << (ue, float(t))
        #out_file_ubar << (u_, float(t))
        
# ------------ STEP 2: Helmholtz filter ----------------------#
    
        if N == 0:
            u_tilde0 = deconv_filter(u_, delta, Q, boundary)
            DF = Expression('u_tilde0', degree = degree, u_tilde0 = u_tilde0)

        if N == 1:
            u_tilde0 = deconv_filter(u_, delta, Q, boundary)
            u_tilde1 = deconv_filter(u_tilde0, delta, Q, boundary)
            DF = Expression('2.0*u_tilde0 - u_tilde1', degree = degree, u_tilde0 = u_tilde0, u_tilde1 = u_tilde1)

        if N == 2:
            u_tilde1 = deconv_filter(u_, delta, Q, boundary)
            u_tilde2 = deconv_filter(u_tilde1, delta, Q, boundary)
            u_tilde3 = deconv_filter(u_tilde2, delta, Q, boundary)
            DF = Expression('3.0*u_tilde1 - 3.0*u_tilde2 + u_tilde3', degree = degree, u_tilde1 = u_tilde1, u_tilde2 = u_tilde2, u_tilde3 = u_tilde3)

        if N == 3:
            u_tilde1 = deconv_filter(u_, delta, Q, boundary)
            u_tilde2 = deconv_filter(u_tilde1, delta, Q, boundary)
            u_tilde3 = deconv_filter(u_tilde2, delta, Q, boundary)
            u_tilde4 = deconv_filter(u_tilde3, delta, Q, boundary)
            DF = Expression('3.0*u_tilde1 - 3.0*u_tilde2 + u_tilde3 + u_tilde1 - 3.0*u_tilde2 + 3.0*u_tilde3 - u_tilde4', degree = degree, u_tilde1 = u_tilde1, u_tilde2 = u_tilde2, u_tilde3 = u_tilde3, u_tilde4 = u_tilde4)
            
        # Compute the indicator function N = 0
        indicator = Expression('sqrt((a-b)*(a-b))', degree = 2, a = u_, b = DF)
        indicator = interpolate(indicator, Q)
        max_ind = np.amax(indicator.vector().get_local())

        if max_ind < 1:
            max_ind = 1.0

        indicator = Expression('a/b', degree = 2, a = indicator, b = max_ind)
        indicator = interpolate(indicator, Q)

        indicator.rename('a','a')
        #out_file_ind << (indicator, t)

        # Apply the filter
        u_bar = TrialFunction(Q)
        F_filter = v*u_bar*dx + delta*delta*dot(grad(v), indicator*grad(u_bar))*dx - v*u_*dx 

        a_filter = lhs(F_filter)
        L_filter = rhs(F_filter)

        A_filter = assemble(a_filter)
        bc.apply(A_filter)

        b_filter = assemble(L_filter)
        bc.apply(b_filter)

        solver = LUSolver(A_filter)
        u_bar = Function(Q)
        solver.solve(u_bar.vector(), b_filter)

        u_bar.rename('u','u')
#         out_file_ufilter << (u_bar, float(t))

        
        u_n.assign((1 - chi)*u_+ chi*u_bar)
        
    out_file_ind << (indicator, float(t))
    out_file_ufilter << (u_bar, float(t))   
    L2, H1 = compute_errors(u_exact, u_n, t, mesh)
    maxval, minval = compute_extrema(u_n, t)
    
    print(nx,",",L2,",",H1,",",maxval,",",minval)
    return str(L2)+','+str(H1)+','+str(maxval)+','+str(minval)
    
# ------------------------------------------ #

from datetime import datetime, date

def run(nx,chi,dt,N):
    delta = 1.0/nx;
    d_date = datetime.now()
    today = d_date.strftime('%Y-%m-%d')
    f = open(today+'_log_EFR.txt',"a+")


    T = 0.5
    degree = 2

    sim_data = 'EFR, dt='+str(dt)+', T='+str(T)+', P'+str(degree)
    f.write(sim_data+'\n')


    now = d_date.strftime("%I:%M:%S %p")
    f.write('start: '+now+'\n\n')

    remark = ''
    output = EF_t(remark, N, delta, nx, dt, T, u_code, f_code, sigma, mu, b, chi)
    f.write(str(nx)+','+str(dt)+','+output+'\n')


    d_date = datetime.now()
    now = d_date.strftime("%I:%M:%S %p")
    f.write('\nend: '+now+'\n\n')
    f.write(notes)

    f.close()

In [2]:
dt = 0.1
N = 0
# nx,chi,dt,N
run(25,1,dt,N)
dt = dt/2
run(50,0.5,dt,N)
dt = dt/2
run(100,0.25,dt,N)
dt = dt/2
run(200,1.0/16,dt,N)
dt = dt/2
run(400,1.0/256,dt,N)

25 , 0.101644454499 , 7.11646285792 , 1.14335835147 , -0.325484898114
50 , 0.0502750153797 , 10.1177091045 , 1.10109750818 , -0.21010603197
100 , 0.0191676896858 , 8.66135384127 , 1.08311299003 , -0.0976072708457
200 , 0.00376883513513 , 3.08442698565 , 1.0015173753 , -0.018750656366
400 , 0.00136814395663 , 0.813711694729 , 0.990251110774 , -0.00518403910566


In [3]:
dt = 0.1
N = 1
# nx,chi,dt,N
run(25,1,dt,N)
dt = dt/2
run(50,0.5,dt,N)
dt = dt/2
run(100,0.25,dt,N)
dt = dt/2
run(200,1.0/16,dt,N)
dt = dt/2
run(400,1.0/256,dt,N)

25 , 0.102119537721 , 7.15293291842 , 1.14487679373 , -0.347378716601
50 , 0.0464783464538 , 6.09192693364 , 1.02870388218 , -0.144570356624
100 , 0.0178457985483 , 4.57493554413 , 1.04544454922 , -0.0669257032661
200 , 0.00374445786511 , 2.27458220756 , 0.99822765621 , -0.015116371739
400 , 0.0013827156859 , 0.801814646286 , 0.990250055899 , -0.00486892815668


In [4]:
dt = 0.1
N = 2
# nx,chi,dt,N
run(25,1,dt,N)
dt = dt/2
run(50,0.5,dt,N)
dt = dt/2
run(100,0.25,dt,N)
dt = dt/2
run(200,1.0/16,dt,N)
dt = dt/2
run(400,1.0/256,dt,N)

25 , 0.102231745362 , 7.15773287544 , 1.14345362481 , -0.355078904732
50 , 0.0465815240431 , 6.16708237005 , 1.03150271491 , -0.146812341539
100 , 0.0178094826995 , 4.64651800513 , 1.04660901315 , -0.0685466039256
200 , 0.00371032444576 , 2.29156552423 , 0.998299737706 , -0.0151994027019
400 , 0.00137323378281 , 0.777418157328 , 0.990250104136 , -0.0048835335912


In [5]:
dt = 0.1
N = 3
# nx,chi,dt,N
run(25,1,dt,N)
dt = dt/2
run(50,0.5,dt,N)
dt = dt/2
run(100,0.25,dt,N)
dt = dt/2
run(200,1.0/16,dt,N)
dt = dt/2
run(400,1.0/256,dt,N)

25 , 0.102277825225 , 7.16713516004 , 1.14523067146 , -0.36030503622
50 , 0.0466774204674 , 6.22605046432 , 1.03441131425 , -0.148388758563
100 , 0.017769392088 , 4.69392929983 , 1.04745455677 , -0.0696024117064
200 , 0.00368994746152 , 2.30499109931 , 0.998365374756 , -0.0152577255162
400 , 0.00136956936154 , 0.770475464263 , 0.990250136419 , -0.00489327705034


In [6]:
dt = 0.05
N = 3
# nx,chi,dt,N
run(25,1,dt,N)
dt = dt/2
run(50,0.5,dt,N)
dt = dt/2
run(100,0.25,dt,N)
dt = dt/2
run(200,1.0/16,dt,N)
dt = dt/2
run(400,1.0/256,dt,N)

25 , 0.098041046975 , 7.03659821951 , 1.17060827 , -0.330009806246
50 , 0.0408284833069 , 5.80358562722 , 1.02638245472 , -0.1313289547
100 , 0.0157853636451 , 4.38245777155 , 1.04442112872 , -0.0600755945281
200 , 0.00271338556835 , 2.14593778909 , 0.999478770609 , -0.0117512589845
400 , 0.00070110239031 , 0.559146153499 , 0.991124344793 , -0.00259002385853


In [7]:
dt = 0.01
N = 3
# nx,chi,dt,N
run(25,1,dt,N)
dt = dt/2
run(50,0.5,dt,N)
dt = dt/2
run(100,0.25,dt,N)
dt = dt/2
run(200,1.0/16,dt,N)
dt = dt/2
run(400,1.0/256,dt,N)

25 , 0.0977218205311 , 6.92748030358 , 1.10629039331 , -0.297447903134
50 , 0.0353500631391 , 5.19036479927 , 1.00091140221 , -0.105583128981
100 , 0.013128227199 , 3.60329864608 , 1.02649779969 , -0.0402344886448
200 , 0.00221450340804 , 1.94306477512 , 0.999292809638 , -0.00862977066395
400 , 0.000210987824156 , 0.462462906361 , 0.991823997103 , -0.000638926977138


In [8]:
dt = 0.05
N = 2
# nx,chi,dt,N
run(25,1,dt,N)
dt = dt/2
run(50,0.5,dt,N)
dt = dt/2
run(100,0.25,dt,N)
dt = dt/2
run(200,1.0/16,dt,N)
dt = dt/2
run(400,1.0/256,dt,N)

25 , 0.0979907904393 , 7.03085563211 , 1.16700451567 , -0.326438175463
50 , 0.0408245024651 , 5.75841621535 , 1.02328155719 , -0.129836787826
100 , 0.0158342518114 , 4.33666397624 , 1.04364110733 , -0.0589362734199
200 , 0.00274142259994 , 2.13262151424 , 0.999416600261 , -0.0117373135606
400 , 0.000708204491794 , 0.569286169269 , 0.991124316058 , -0.00258690727223


In [9]:
dt = 0.01
N = 2
# nx,chi,dt,N
run(25,1,dt,N)
dt = dt/2
run(50,0.5,dt,N)
dt = dt/2
run(100,0.25,dt,N)
dt = dt/2
run(200,1.0/16,dt,N)
dt = dt/2
run(400,1.0/256,dt,N)

25 , 0.0981503015328 , 6.92061303111 , 1.10075795866 , -0.297221779469
50 , 0.0360894574575 , 5.17902449929 , 0.996782919159 , -0.105553587711
100 , 0.0134052939904 , 3.56753888425 , 1.02551762076 , -0.0382530473193
200 , 0.00226843607399 , 1.92980042364 , 0.999215769955 , -0.00859647515042
400 , 0.000233440073069 , 0.474742397549 , 0.991823910267 , -0.000638995713215


In [10]:
dt = 0.05
N = 1
# nx,chi,dt,N
run(25,1,dt,N)
dt = dt/2
run(50,0.5,dt,N)
dt = dt/2
run(100,0.25,dt,N)
dt = dt/2
run(200,1.0/16,dt,N)
dt = dt/2
run(400,1.0/256,dt,N)

25 , 0.0976925107096 , 7.01653874265 , 1.16530645102 , -0.32141150826
50 , 0.0408437682842 , 5.69895446425 , 1.02008246217 , -0.127971675906
100 , 0.015878273991 , 4.26495672473 , 1.04248542658 , -0.0571585278331
200 , 0.00278807297222 , 2.11566875225 , 0.999350127489 , -0.0117384593934
400 , 0.000726299589738 , 0.602842436974 , 0.991124260968 , -0.00258263384607


In [11]:
dt = 0.01
N = 1
# nx,chi,dt,N
run(25,1,dt,N)
dt = dt/2
run(50,0.5,dt,N)
dt = dt/2
run(100,0.25,dt,N)
dt = dt/2
run(200,1.0/16,dt,N)
dt = dt/2
run(400,1.0/256,dt,N)

25 , 0.0987410976511 , 6.9066919818 , 1.09107367773 , -0.296629815575
50 , 0.0371746522025 , 5.1571411354 , 0.994741782732 , -0.103697080431
100 , 0.0138100055043 , 3.50881756328 , 1.02405038442 , -0.0357443232471
200 , 0.00236142188029 , 1.91322656533 , 0.999117856529 , -0.00856595902489
400 , 0.000283578853244 , 0.514664109563 , 0.991823716208 , -0.000639107430288


In [12]:
dt = 0.05
N = 0
# nx,chi,dt,N
run(25,1,dt,N)
dt = dt/2
run(50,0.5,dt,N)
dt = dt/2
run(100,0.25,dt,N)
dt = dt/2
run(200,1.0/16,dt,N)
dt = dt/2
run(400,1.0/256,dt,N)

25 , 0.0970391726477 , 6.96714467677 , 1.15397913842 , -0.308335430484
50 , 0.0406453258565 , 5.56977742134 , 1.0141133774 , -0.124640923352
100 , 0.0158473198451 , 4.10707358966 , 1.03984611105 , -0.054521349367
200 , 0.00287273843978 , 2.06715929327 , 0.9991773173 , -0.0117741651093
400 , 0.000787433076104 , 0.706725066234 , 0.991123604711 , -0.00257495385563


In [13]:
dt = 0.01
N = 0
# nx,chi,dt,N
run(25,1,dt,N)
dt = dt/2
run(50,0.5,dt,N)
dt = dt/2
run(100,0.25,dt,N)
dt = dt/2
run(200,1.0/16,dt,N)
dt = dt/2
run(400,1.0/256,dt,N)

25 , 0.0995468866622 , 6.87791216655 , 1.04590951257 , -0.273544672495
50 , 0.0387092138865 , 5.07127946639 , 0.989641191412 , -0.0901757345626
100 , 0.0143665578461 , 3.35308379005 , 1.01986239936 , -0.0352645938854
200 , 0.00253781746279 , 1.85926063107 , 0.998779445927 , -0.00917650988178
400 , 0.000415920966288 , 0.633763178961 , 0.99182264811 , -0.000640531728557
